# WINDNODE ABW - Scenario Analysis

<img src="http://reiner-lemoine-institut.de//wp-content/uploads/2015/09/rlilogo.png" width="100" style="float: right">

__copyright__ 	= "© Reiner Lemoine Institut" <br>
__license__ 	= "GNU Affero General Public License Version 3 (AGPL-3.0)" <br>
__url__ 		= "https://www.gnu.org/licenses/agpl-3.0.en.html" <br>
__author__ 		= "Julian Endres" <br>

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
######## WINDNODE ###########
# define and setup logger
from windnode_abw.tools.logger import setup_logger
logger = setup_logger()

# load configs
from windnode_abw.tools import config
config.load_config('config_data.cfg')
config.load_config('config_misc.cfg')

from windnode_abw.analysis import analysis
from windnode_abw.tools.draw import *

######## DATA ###########

import re
import pandas as pd

######## Plotting ###########

# Plotting
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import ScalarFormatter
import seaborn as sns
# set seaborn style
sns.set()

import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
#import plotly.offline as pltly
#import plotly.io as pio

In [ ]:
# specify what to import (in path ~/.windnode_abw/)
run_timestamp = '2020-07-24_104145_1month'
scenario = 'NEP2035'

In [ ]:
# obtain processed results
regions_scns, results_scns = analysis(run_timestamp=run_timestamp,
                                      scenarios=[scenario])

# TODO
- [ ] parallelize analysis
- [ ] save / pickle results

### useful params

In [ ]:
MUN_NAMES = regions_scns[scenario].muns.gen.to_dict()
bar_colors = px.colors.sequential.GnBu_r
UNITS = {"relative": "%", "hours": "h",}

# 1 Demand and Generation (Input Data)

## 1.1 Installed Electrical Capacities

In [ ]:
df_data = results_scns[scenario]['parameters']['Installed capacity electricity supply']
df_data = df_data.rename(columns=PRINT_NAMES)

fig, axes = plt.subplots(3,3, figsize=(12,10))
for ax, (key, data) in  zip(axes.flat, df_data.iteritems()):
    plot_geoplot(key, data, regions_scns[scenario], ax=ax, unit='MW')
    
fig.suptitle('Installed el. Generation Capacity',
     fontsize=16,
     fontweight='normal')
plt.tight_layout()
plt.show()

## 1.2 Electrical Demand

In [ ]:
df_data = results_scns[scenario]['flows_txaxt']['Stromnachfrage'].sum(level=1)
df_data = df_data.drop(columns='export')
df_data.index = df_data.index.astype(int)
df_data = df_data.rename(columns=PRINT_NAMES)
df_data = df_data / 1e3

fig, axes = plt.subplots(1,3, figsize=(14,4))
for ax, (key, data) in  zip(axes.flat, df_data.iteritems()):
    plot_geoplot(key, data, regions_scns[scenario], ax=ax, unit='GWh')

fig.suptitle('Electrical Demand per Sector',
     fontsize=16,
     fontweight='normal')
plt.tight_layout()
plt.show()

## 1.3 Thermal Demand

In [ ]:
df_data = results_scns[scenario]['flows_txaxt']['Wärmenachfrage'].sum(level=2)
df_data.index = df_data.index.astype(int)
df_data = df_data.rename(columns=PRINT_NAMES)
df_data = pd.DataFrame([df_data.iloc[:,:2].sum(axis=1).rename('Households'), df_data.iloc[:,-1]]).T
df_data = df_data / 1e3

fig, axes = plt.subplots(1,2, figsize=(14,4))
for ax, (key, data) in  zip(axes.flat, df_data.items()):
    plot_geoplot(key, data, regions_scns[scenario], ax=ax, unit='GWh')
    
fig.suptitle('Thermal Demand per Sector',
     fontsize=16,
     fontweight='normal')
plt.tight_layout()
plt.show()

# 2 Area required by RES

## 2.1 Absolute Area

In [ ]:
df_data = results_scns[scenario]['results_axlxt']['Area required']
df_data = df_data.rename(columns=PRINT_NAMES)

fig, axes = plt.subplots(2,2, figsize=(12,6))
for ax, (key, data) in  zip(axes.flat, df_data.iteritems()):
    plot_geoplot(key, data, regions_scns[scenario], ax=ax, unit='ha')
    
fig.suptitle('Required Area',
     fontsize=16,
     fontweight='normal')
plt.tight_layout()
plt.show()

## 2.2 Relative Area

The following figure shows the total relative area required by the RES in ABW for the current scenario compared to the available areas for different land use scenarios, per RES technology.

Notes:
- The current scenarios of wind turbines and ground-mounted PV is marked with "(current)"
- Technology-specific naming conventions of land use scenarios
  - Wind: Distance to settlements (500m/1000m), use of forests (with/without), percentage of available area due to restrictions resulting from case-by-case decisions
  - PV ground: Restrictions that apply (hard: H / hard+weak: HS), percentage of total available agricultural area

In [ ]:
df_data = results_scns[scenario]['highlevel_results']

fig = go.Figure()

# PV rooftop
mask = [i for i in df_data.index if 'PV rooftop' in  i[0]]
data = df_data.loc[mask]
index = data.index.get_level_values(level=0)

fig.add_trace(
    go.Bar(y=index, x=data.values,
           orientation='h',
           name='PV rooftop',
           marker_color=bar_colors[5]))

# PV Ground
mask = [i for i in df_data.index if 'PV ground' in  i[0]]
data = df_data.loc[mask]
index = data.index.get_level_values(level=0)

fig.add_trace(
    go.Bar(y=index, x=data.values,
           orientation='h',
           name='PV Ground',
           marker_color=bar_colors[4]))#, visible='legendonly'))

# Wind
mask = [i for i in df_data.index if 'rel. wind' in  i[0]]
data = df_data.loc[mask]
index=  data.index.get_level_values(level=0)

fig.add_trace(
    go.Bar(y=index, x=data.values,
           orientation='h',
           name='Wind',
          marker_color=bar_colors[3]))#, visible='legendonly'))

fig.update_layout(title_text = 'Relative Required Area',
                  xaxis=dict(title=' %',
                    titlefont_size=12),
                    autosize=True)

fig.show()

# 3 Electrical Autarky

Autarky describes how much of the demanded energy can be supplied from sources within the region/municipality with regard to energy balance (annual sum) or simultaneity (number of hours).

- [ ] histogramm timeseries
- [ ] boxplots timeseries (prozentual Energiebilanz)

## 3.1 Supply/Demand

In [ ]:
df_data = results_scns[scenario]['results_axlxt']['Autarky']
df_data = df_data.rename(index=MUN_NAMES)
df_data = df_data.sort_values(axis=0, by='relative')

fig = make_subplots(rows=1, cols=2, horizontal_spacing=0.17, column_widths=[0.8, 0.2],
                    specs=[[{"secondary_y": True}, {"secondary_y": True}]])

# === municipalities ===
# supply per municipality
data = df_data['supply'] / 1e3

fig.add_trace(
    go.Bar(x=data.index, y=data.values,
           orientation='v',
           name='Supply',
          marker_color=bar_colors[1],),
    row=1, col=1, secondary_y=False,)

# demand per municipality
data = df_data['demand'] / 1e3

fig.add_trace(
    go.Bar(x=data.index, y=data.values,
           orientation='v',
           name='Demand',
           marker_color=bar_colors[2],),
    row=1, col=1, secondary_y=False,)

# relative autarky per municipality
data = df_data['relative'] * 1e2

fig.add_trace(
    go.Bar(x=data.index, y=data.values,
           orientation='v',
           name='Relative',
           marker_color=bar_colors[3],
           visible='legendonly',
          opacity=0.7,),
    row=1, col=1, secondary_y=True,)

# === ABW ===
df_data = results_scns[scenario]['results_t']['Autarky']

fig.add_trace(
    go.Bar(x=['supply'], y=[df_data['supply']/ 1e3],
           orientation='v',
           name='ABW Supply',
           marker_color=bar_colors[1]), 
    row=1, col=2, secondary_y=False,)

fig.add_trace(
    go.Bar(x=['demand'], y=[df_data['demand'] / 1e3],
           orientation='v',
           name='ABW Demand',
           marker_color=bar_colors[2],),
    row=1, col=2, secondary_y=False,)

fig.add_trace(
    go.Bar(x=[df_data.name], y=[df_data['relative']* 1e2],
           orientation='v',
           name='ABW relative',
           marker_color=bar_colors[3],
           visible='legendonly',),
    row=1, col=2, secondary_y=True,)

# === Layout ===
fig.update_layout(title_text = 'Electrical Autarky per Municipality (Energy Balance) and total region',
                    autosize=True,
                hovermode="x unified",
                  legend=dict(orientation="h",
                                yanchor="bottom",
                                y=1.02,
                                xanchor="right",
                                x=1),
                 )
fig.update_yaxes(title_text="absolute in GWh", row=1, col=1, anchor="x", secondary_y=False)
fig.update_yaxes(title_text="relative in %", row=1, col=1, anchor="x", secondary_y=True)
fig.update_yaxes(title_text="absolute in GWh", row=1, col=2, anchor="x2", secondary_y=False)
fig.update_yaxes(title_text="relative in %", row=1, col=2, anchor="x2", secondary_y=True)
fig.show()

## 3.2 Relative Autarky

In [ ]:
df_data = results_scns[scenario]['results_axlxt']['Autarky']['relative']
df_data = df_data.rename(index=MUN_NAMES)
df_data = df_data.mul(100)
df_data = df_data.sort_values()

data=df_data.round()

limit=120

# split data
data_left = data[data < limit]
data_right = data[data >= limit]

fig = make_subplots(rows=1, cols=2, horizontal_spacing=0.2)

fig.add_trace(
    go.Bar(y=data.index, x=data_left.values,
           orientation='h', name=f'< {limit} %',
          marker_color=bar_colors[1]),
    row=1, col=1)

fig.add_trace(
    go.Bar(y=data.index, x=data_right.values,
           orientation='h', name=f'> {limit} %',
          marker_color=bar_colors[1]),
    row=1, col=2)

fig.update_yaxes(type='category', row=1, col=1)
fig.update_yaxes(type='category', row=1, col=2)
fig.update_xaxes(title='%', anchor='x', row=1, col=1)
fig.update_xaxes(title='%', anchor='x2', row=1, col=2)

fig.update_layout(title_text='Grouped Electrical Autarky per Municipality (Energy Balance)',
                  xaxis=dict(title=' %',
                    titlefont_size=12),
                    autosize=True)
fig.show()

## 3.3 relative time

- [ ] Summe über alle Gemeinde als TS zusätzlich?
- [ ] relativer Zeitliche Anteil Autarky 10% aller stunden?? was habe ich hiermit gemeint?

In [ ]:
df_data = results_scns[scenario]['results_axlxt']['Autarky'][['hours']]
max_hours  = len(results_scns[scenario]['flows_txaxt']['Autarky'].index.get_level_values(level=0).unique())
df_data = df_data['hours'] / max_hours * 1e2

df_data = df_data.sort_values(ascending=False)
df_data = df_data.rename(index=MUN_NAMES)

fig = go.Figure()
fig.add_trace(
    go.Bar(x=df_data.index,
           y=df_data.values,
           orientation='v',
           marker_color=bar_colors[1]))

fig.update_yaxes(title='%')
fig.update_layout(
    title='Percentage of Autark Hours', 
    showlegend=False)
fig.show()

## 3.4 Autarky Geoplots

In [ ]:
df_data = results_scns[scenario]['results_axlxt']['Autarky'].loc[:,['hours', 'relative']]
df_data = df_data.rename(columns=PRINT_NAMES)

fig, axes = plt.subplots(1,2, figsize=(12,5))

for ax, (key, data) in  zip(axes.flat, df_data.iteritems()):
    
    plot_geoplot(key, data, regions_scns[scenario], ax=ax, unit=UNITS[key])
    
fig.suptitle('Autark Hours and Relative Autarky Balance',
     fontsize=16,
     fontweight='normal')
plt.tight_layout()
plt.show()

## 3.5 violinplot

In [ ]:
df_data = results_scns[scenario]['flows_txaxt']['Autarky']['relative'].unstack()

df_data.columns = [int(i) for i in df_data.columns]
df_data = df_data.rename(columns=MUN_NAMES)

limit = 2

fig = make_subplots(rows=2, cols=2, horizontal_spacing=0.25,
                    row_heights=[0.8, 0.2],
                    specs=[[{}, {}], [{"colspan": 2}, None]],)

for ags, data in df_data.iteritems():
    
    if data.describe().loc['mean'] > limit:

        fig.add_trace(go.Violin(x=data.values, name=ags,
                                orientation='h',
                               line_color=bar_colors[1],
                               showlegend=False),
                              
                      row=1, col=1)
    else:
        fig.add_trace(go.Violin(x=data.values, name=ags,
                                orientation='h',
                               line_color=bar_colors[1],
                               showlegend=False),
                      row=1, col=2)
        
    fig.add_trace(go.Violin(x=data.values, name=ags,
                                orientation='h',
                               line_color=bar_colors[1],
                               showlegend=True,
                           visible='legendonly'),
                              
                      row=2, col=1)
    
fig.update_xaxes(title_text="%", side='bottom', row=2, col=1)
fig.update_yaxes(type='category', row=1, col=1)
fig.update_yaxes(type='category', row=1, col=2)

fig.update_layout(
    title='Relative Electrical Autarky per Municipality (Energy Balance)')
fig.show()

# 4 Energy Mix

INFO:
- export == ins nationale Netz
- ABW_... == intra_regional

TODO:

## 4.1 Balance

In [ ]:
supply = results_scns[scenario]['flows_txaxt']['Stromerzeugung'].sum(level=1)
abw_import = results_scns[scenario]['flows_txaxt']['Intra-regional exchange']['import'].sum(level=1)
abw_import = abw_import.rename('ABW-import')
supply = supply.join(abw_import)


demand = results_scns[scenario]['flows_txaxt']['Stromnachfrage'].sum(level=1)
abw_export = results_scns[scenario]['flows_txaxt']['Intra-regional exchange']['export'].sum(level=1)
abw_export = abw_export.rename('ABW-export')
demand = demand.join(abw_export)
el_heating = results_scns[scenario]['flows_txaxt']['Stromnachfrage Wärme'].sum(level=2).sum(axis=1)
demand = demand.join(el_heating.rename('el_heating'))
    
plot_snd_total(regions_scns[scenario], supply , demand)

## 4.2 Time

In [ ]:
plot_timeseries(results_scns[scenario], kind='el')#, ags='15091160')

# 5 Emissions

TODO
- [ ] große Werte besonders schöne Farben
- [ ] Notiz zur Zuordnung von KWK zu el etc

In [ ]:
df_data_left = results_scns[scenario]['results_t']['CO2 emissions th. total'].rename('th').to_frame()
df_data_right = results_scns[scenario]['results_t']['CO2 emissions el. total'].rename('el').to_frame()

# drop nans & zeros
df_data_left = df_data_left[df_data_left!=0].dropna()
df_data_right = df_data_right[df_data_right!=0].dropna()

df_data = df_data_left.join(df_data_right, how='outer')
df_data = df_data.fillna(0).T
df_data = df_data.sort_values(by=list(df_data.index), axis=1, ascending=True)
#df_rel = df_data.T / df_data.sum(axis=1).values

fig = px.bar(df_data, orientation='h',
             color_discrete_sequence=bar_colors,
             text=df_data.sum(axis=1).to_list()
            )


fig.update_layout(barmode='stack',
                  autosize=True,
                  title='CO2 Emissions',
                  legend={'traceorder':'reversed'},
                 uniformtext_mode='hide')
fig.update_traces(hovertemplate='CO2: %{x:.1f} t <br>Type: %{y}<br>Total: %{text:.1f} t') # 
fig.update_xaxes(title_text='t CO2')
fig.update_yaxes(title_text='')
fig.show()

# 6 Costs

## 6.1 LCOE and LCOH

<div class="alert alert-block alert-info">
<b>Notes on LCOE calculation</b>

- Total LCOE calculate as $LCOE=\frac{expenses_{el.total}}{demand_{el.,total}}$, likewise total LCOH calculate as $LCOH=\frac{expenses_{th.,total}}{demand_{th.,total}}$
- Total expenses $expenses_{el.total}$ are annual expenses. Investment costs are discounted to one year using equivalent periodic costs
- The plot below shows fractions of these LCOE that are calculated as $LCOE_{technology}=\frac{expenses_{el.,technology}}{demand_{el.,total}}$ representation the share of each technology at total cost of one MWh
</div>

In [ ]:
values = ['LCOE','LCOH']

df = pd.DataFrame([results_scns[scenario]['results_t'][i] for i in values], index=values)
df = df.rename(columns=PRINT_NAMES)
df = df.sort_values(by=values, axis=1, ascending=True)

fig = px.bar(df, orientation='h',
             title='LCOE and LCOH',
            color_discrete_sequence=bar_colors,
                text=df.sum(axis=1).to_list())

fig.update_layout(barmode='stack', legend={'traceorder':'reversed'},
                  uniformtext_mode='hide',
                 )
fig.update_traces(hovertemplate='Share: %{x:.1f}€ <br>Type: %{y}<br>Total: %{text:.1f}€'  ) # 
fig.update_xaxes(title_text='€/MWh')
fig.update_yaxes(title_text='')
fig.show()

# 7 Power Grid

In [ ]:
df_data = results_scns[scenario]['flows_txaxt']['Line loading'].max(level=1) * 100
df_data = df_data.sort_index(ascending=False)
df_data = pd.DataFrame().from_dict({'line loading': df_data,'free capacity': 60-df_data})

index = [re.split(r'(\d+)', s) for s in df_data.index]
df_data.index = [f"{start}{MUN_NAMES[int(ags)]}{end}" for start,ags,end in index]

fig = go.Figure() 
for i, (key, df) in enumerate(df_data.items()):

    fig.add_bar(y=df.index,
                x=df.values,
                orientation='h',
                name=key,
                marker_color=bar_colors[4*i+2],
                hovertemplate='%{x:.2f}%',
                showlegend=False)

fig.update_layout(barmode="relative",
    title='Maximum Line Loading',
    yaxis_tickfont_size=12,
    xaxis=dict(
        title='Loading in %',
        titlefont_size=16,
        tickfont_size=12,
    ))

fig.update_yaxes(type='category')
fig.update_layout(width=800, hovermode="y unified")
fig.show()


In [484]:
df_data = results_scns[scenario]['flows_txaxt']['Line loading per bus']
df_data


x = df_data.index.get_level_values(level=0)
#x = pd.Series(x.astype(int).values).map(MUN_NAMES).values
fig = go.Figure()

fig.add_trace(go.Box(
    y=df_data.values * 1e2,
    x=x,
    name='Increase',
    marker_color='#3D9970',
    boxpoints=False,

))

fig.update_layout(
    title='Line Loading',
    yaxis_title='in %',
    boxmode='group'
)

fig.update_xaxes(type='category', tickangle=45)
fig.show()


In [ ]:
# violinplots
df_data = results_scns[scenario]['flows_txaxt']['Autarky']['relative'].unstack()

df_data.columns = [int(i) for i in df_data.columns]
df_data = df_data.rename(columns=MUN_NAMES)

limit = 2

fig = make_subplots(rows=1, cols=2, horizontal_spacing=0.25)

for ags, data in df_data.iteritems():
    
    if data.describe().loc['mean'] > limit:

        fig.add_trace(go.Violin(x=data.values, name=ags, orientation='h'), row=1, col=1)
    else:
        fig.add_trace(go.Violin(x=data.values, name=ags,  orientation='h'), row=1, col=2)


fig.update_xaxes(title_text="% autarky", row=1, col=1)
fig.update_yaxes(type='category', row=1, col=1)
fig.update_yaxes(type='category', row=1, col=2)


fig.update_layout(
    title='Relative Electrical Autarky per Municipality (Energy Balance)')
fig.show()

# ENDE

# advanced/split violin plot

# 8 Flexibility

## Heatstorage

Fragen:
- ist usage überhaupt der Nutzungsgrad? jain

- [ ] Kennzahlen erstellen
    - Charge / Ladeleistung (nominal value)
    - Discharge / Entladeleistung
    - Charge / Nennspeicherkapazität
    - Discharge / Nennspeicherkapazität

## Batterystorage

- Batteriespeicher leer?

- nur increase activation?

## DSM

- Bezug auf hh-demand-mit-dsm beziehen?
    - auf total
    - auf hh-demand
- absolute dsm activation
- pro AGS

- nur increase activation? JA!

# inter ags elec transport / chord diagramm

## chord diagram

- Plotly lässt sich nicht als bereits berechnetes HTML anzeigen / nur zur veranschaulichung
- -> Binder

#### Darstellung mit matplotlib

- Summe über Gemeinde als TS zusätzlich
- relativer Zeitliche Anteil Autarky
- mehrere Plots für ags?
- Summen über Tage?

- geoplot